In [1]:
import tensorflow as tf
import os
import pickle as p
import numpy as np 

CIFAR_DIR = "./cifar-10-batches-py"
print(os.listdir(CIFAR_DIR))

['data_batch_5', 'batches.meta', 'data_batch_4', 'data_batch_3', 'data_batch_1', 'readme.html', 'data_batch_2', 'test_batch']


In [2]:
# tensorboard
# 1. 指定面板图上显示的变量
# 2. 训练过程中将这些变量计算出来，输出到文件中
# 3. 文件解析 ./ensorboard --logdir=dir.

# batch normalization: 需要在每个卷积后添加

In [3]:
def load_data(filename):
    """read data from data file"""
    with open(filename,'rb') as f:
        data = p.load(f,encoding='bytes')
        return data[b'data'],data[b'labels']
    
class CifarData:
    def __init__(self,filenames,need_shuffle):
        all_data = []
        all_labels = []
        for filename in filenames:
            data, labels = load_data(filename)
            all_data.append(data)
            all_labels.append(labels)
        self._data = np.vstack(all_data)  #纵向合并成一个矩阵
        self._labels = np.hstack(all_labels)  #横向合并～
        print(self._data.shape)
        print(self._labels.shape)
  
        self._num_examples = self._data.shape[0]  #样本数目
        self._need_shuffle = need_shuffle   
        #样本数据是否需要打乱，是否可以重复使用数据样本
        self._indicator = 0  #表明当前数据集已经定位到哪个位置了
        if self._need_shuffle:
            self._shuffle_data()
            
    def _shuffle_data(self):
        #顺序打乱  将0～(num-1)的数混乱排序并输出
        p = np.random.permutation(self._num_examples)
        self._data = self._data[p]
        self._labels = self._labels[p]
        
    def next_batch(self,batch_size):
        """return batch_size example as a batch"""
        end_indicator = self._indicator + batch_size #定义结束位置
        if end_indicator > self._num_examples:  
            if self._need_shuffle:
                self._shuffle_data()
                self._indicator = 0
                end_indicator = batch_size
            else:
                raise Exception("have no more example")
        if end_indicator > self._num_examples:
        #这个判断batch_size是不是大于样本总数
            raise Exception("batch size is larger than all examples")
        batch_data = self._data[self._indicator:end_indicator]
        batch_labels = self._labels[self._indicator:end_indicator]
        self._indicator = end_indicator
        return batch_data,batch_labels
    
train_filenames = [os.path.join(CIFAR_DIR,'data_batch_%d' % i) for i in range(1,6)]
test_filenames = [os.path.join(CIFAR_DIR,'test_batch')]

train_data = CifarData(train_filenames,True)
test_data = CifarData(test_filenames,False)

# batch_data, batch_labels = train_data.next_batch(10)
# print(batch_data)
# print(batch_labels)

(50000, 3072)
(50000,)
(10000, 3072)
(10000,)


In [4]:
batch_size=20
x = tf.placeholder(tf.float32,[batch_size,3072])
y = tf.placeholder(tf.int64,[batch_size])
is_training = tf.placeholder(tf.bool, [])

x_image = tf.reshape(x, [-1, 3, 32, 32])
# 32*32
x_image = tf.transpose(x_image, perm=[0, 2, 3, 1])

x_image_arr = tf.split(x_image,
                       num_or_size_splits = batch_size,
                       axis = 0)
result_x_image_arr = []
for x_single_image in x_image_arr:
    # x_single_image: [1, 32, 32, 3] -> [32, 32, 3]
    x_single_image = tf.reshape(x_single_image, [32, 32, 3])  
    data_aug_1 = tf.image.random_flip_left_right(x_single_image)
    data_aug_2 = tf.image.random_brightness(data_aug_1, max_delta=63)
    data_aug_3 = tf.image.random_contrast(
        data_aug_2, lower=0.2, upper=1.8)
    x_single_image = tf.reshape(data_aug_3, [1, 32, 32, 3])
    result_x_image_arr.append(x_single_image)
result_x_images = tf.concat(result_x_image_arr, axis=0)
normal_result_x_images = result_x_images / 127.5 -1

# 抽象化卷积
'''
def conv_wrapper(inputs,
                 name,
                 output_channel=32,
                 kernel_size=(3,3),
                 activation = tf.nn.relu,
                 padding = 'same'):
    """wrapper of tf.layers.conv2d"""
    return tf.layers.conv2d(inputs,
                            output_channel,
                            kernel_size,
                            padding = padding,
                            activation = activation,
                            name = name)
'''

def conv_wrapper(inputs,
                 name,
                 is_training,
                 output_channel=32,
                 kernel_size=(3,3),
                 activation = tf.nn.relu,
                 padding = 'same'):
    """wrapper of tf.layers.conv2d"""
    # without bn: conv -> activation
    # with batch normalization: conv -> bn -> activation
    with tf.name_scope(name):
        conv2d = tf.layers.conv2d(inputs,
                                  output_channel,
                                  kernel_size,
                                  padding = padding,
                                  activation = None,
                                  name = name + '/conv2d')
        # bn 在train过程中是用一个batch计算均值和方差，
        # 在test过程中用整个数据集上计算得到
        bn = tf.layers.batch_normalization(conv2d,
                                           training = is_training) # 判断是不是在训练
        return activation(bn)
# 抽象化池化  （wrapper:封套）
def pooling_wrapper(inputs, name):
    """wrapper of tf.layers.max_pooling2d"""
    return tf.layers.max_pooling2d(inputs,
                                   (2,2),
                                   (2,2),
                                   name = name)

#conv1: 神经元图， feature_map，输出图像
conv1_1 = conv_wrapper(normal_result_x_images, 'conv1_1', is_training)
conv1_2 = conv_wrapper(conv1_1, 'conv1_2', is_training)
conv1_3 = conv_wrapper(conv1_2, 'conv1_3', is_training)
pooling1 = pooling_wrapper(conv1_3, 'pool1')

conv2_1 = conv_wrapper(pooling1, 'conv2_1', is_training)
conv2_2 = conv_wrapper(conv2_1, 'conv2_2', is_training)
conv2_3 = conv_wrapper(conv2_2, 'conv2_3', is_training)
pooling2 = pooling_wrapper(conv2_3, 'pool2')

conv3_1 = conv_wrapper(pooling2, 'conv3_1', is_training)
conv3_2 = conv_wrapper(conv3_1, 'comv3_2', is_training)
conv3_3 = conv_wrapper(conv3_2, 'conv3_3', is_training)
pooling3 = pooling_wrapper(conv3_3, 'pool3')

# 展平后变为二维向量 [None, 4 * 4 * 32]
flatten = tf.layers.flatten(pooling3)
y_ = tf.layers.dense(flatten, 10)

# 交叉熵损失
loss = tf.losses.sparse_softmax_cross_entropy(labels = y, logits = y_)
# y_ -> softmax
#y ->one_hot
#loss = ylogy_


# indices
predict = tf.argmax(y_, 1) #在y_的第一维求最大值的索引位置
correct_prediction = tf.equal(predict, y)
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float64))

with tf.name_scope('train_op'):
#     train_op = tf.train.AdamOptimizer(1e-3).minimize(loss)
    optimizer = tf.train.AdamOptimizer(1e-3)
    update_ops = tf.get_collection(tf.GraphKeys.UPDATE_OPS)
    # 有两个方案使用batch_normalization, 第一种如下面的代码使用control_dependencies
    # 第二种是不使用control_dependencies,但在下面训练代码中调用sess.run时把
    # update_ops也加进去，即 sess.run([train_op, update_ops, ..],feed_dict = ..)
    with tf.control_dependencies(update_ops): # 保证train_op在update_ops执行后再执行
        train_op = optimizer.minimize(loss)

In [5]:
def variable_summary(var, name):
    """Constructs summary for statistics of a variable"""
    with tf.name_scope(name):
        mean = tf.reduce_mean(var)
        with tf.name_scope('stddev'):
            stddev = tf.sqrt(tf.reduce_mean(tf.square(var - mean)))
        tf.summary.scalar('mean', mean)
        tf.summary.scalar('stddev', stddev)
        tf.summary.scalar('min', tf.reduce_min(var))
        tf.summary.scalar('max', tf.reduce_max(var))
        tf.summary.histogram('histogram', var)
        
with tf.name_scope('summary'):
    variable_summary(conv1_1, 'conv1_1')
    variable_summary(conv1_2, 'conv1_2')
    variable_summary(conv2_1, 'conv2_1')
    variable_summary(conv2_2, 'conv2_2')
    variable_summary(conv3_1, 'conv3_1')
    variable_summary(conv3_2, 'conv3_2')

loss_summary = tf.summary.scalar('loss', loss)
# 'loss': <10, 1.1>, <20, 1.08>
accuracy_summary = tf.summary.scalar('accuracy', accuracy)

input_summary = tf.summary.image('inputs_image', result_x_images)

merged_summary = tf.summary.merge_all()   #把所有summary合并到一起
merged_summary_test = tf.summary.merge([loss_summary, accuracy_summary])

LOG_DIR = '.'
run_label = './run_VGG_Net_tensorboard'
run_dir = os.path.join(LOG_DIR, run_label)
if not os.path.exists(run_dir):
    os.mkdir(run_dir)
train_log_dir = os.path.join(run_dir, 'train')
test_log_dir = os.path.join(run_dir, 'test')
if not os.path.exists(train_log_dir):
    os.mkdir(train_log_dir)
if not os.path.exists(test_log_dir):
    os.mkdir(test_log_dir)


In [6]:
# model_dir = os.path.join(run_dir, 'model')
# if not os.path.exists(model_dir):
#     os.mkdir(model_dir)
    
# saver = tf.train.Saver(var_list = tf.global_variables())
# model_name = 'ckp-01000'
# model_path = os.path.join(model_dir, model_name)

In [8]:
init = tf.global_variables_initializer()
train_steps = 100000
test_steps = 100

output_summary_every_steps = 100
output_model_every_stpes = 100

# train 100k: 78.05% -> 82.6% -> 83.4% -> 85.6%
with tf.Session() as sess:
    sess.run(init)
    train_writer = tf.summary.FileWriter(train_log_dir, sess.graph)
    test_writer = tf.summary.FileWriter(test_log_dir)
    
    fixed_test_batch_data, fixed_test_batch_labels \
        = test_data.next_batch(batch_size)
    
#     if os.path.exists(model_path + '.index'):
#         saver.restore(sess, model_path)
#         print('model restored from %s' % model_path)
#     else:
#         print('model %s does not exist' % model_path)

    for i in range(train_steps):
        batch_data,batch_labels = train_data.next_batch(batch_size)
        eval_ops = [loss, accuracy, train_op]
        should_output_summary = ((i+1) % output_summary_every_steps == 0)
        if should_output_summary:
            eval_ops.append(merged_summary)
        
        eval_ops_results = sess.run(
            eval_ops,
            feed_dict = {
                x: batch_data, 
                y: batch_labels,
                is_training: True,})
        loss_val, acc_val = eval_ops_results[0:2]
        if should_output_summary:
            train_summary_str = eval_ops_results[-1]
            train_writer.add_summary(train_summary_str, i+1)
            test_summary_str = sess.run([merged_summary_test],
                                        feed_dict={
                                            x: fixed_test_batch_data,
                                            y: fixed_test_batch_labels,
                                            is_training: False,
                                        })[0]
            test_writer.add_summary(test_summary_str, i+1)
            
        if (i+1) % 500 == 0:
            print('[Train] Step: %d, loss: %4.5f, acc: %4.5f' \
                 % (i+1, loss_val, acc_val))
        if (i+1) % 5000 == 0:
            test_data = CifarData(test_filenames,False)
            all_test_acc_val = []
            for j in range(test_steps):
                test_batch_data, test_batch_labels \
                    = test_data.next_batch(batch_size)
                test_acc_val = sess.run(
                    [accuracy],
                    feed_dict = {
                        x: test_batch_data,
                        y: test_batch_labels,
                        is_training: False,})
                all_test_acc_val.append(test_acc_val)
            test_acc = np.mean(all_test_acc_val)
            print('[Test] Step: %d, acc: %4.5f' % (i+1,test_acc))
#         if (i+1) % output_model_every_stpes == 0:
#             saver.save(sess,
#                        os.path.join(model_dir, 'ckp-%05d' % (i+1)))
#             print('model saved to ckp-%05d' % (i+1))

[Train] Step: 500, loss: 1.84517, acc: 0.30000
[Train] Step: 1000, loss: 1.32742, acc: 0.50000
[Train] Step: 1500, loss: 1.57219, acc: 0.30000
[Train] Step: 2000, loss: 1.12057, acc: 0.60000
[Train] Step: 2500, loss: 0.74552, acc: 0.80000
[Train] Step: 3000, loss: 0.64952, acc: 0.75000
[Train] Step: 3500, loss: 1.42837, acc: 0.55000
[Train] Step: 4000, loss: 0.73900, acc: 0.70000
[Train] Step: 4500, loss: 0.57663, acc: 0.80000
[Train] Step: 5000, loss: 1.05474, acc: 0.55000
(10000, 3072)
(10000,)
[Test] Step: 5000, acc: 0.66250
[Train] Step: 5500, loss: 0.56174, acc: 0.70000
[Train] Step: 6000, loss: 0.92993, acc: 0.75000
[Train] Step: 6500, loss: 0.50995, acc: 0.85000
[Train] Step: 7000, loss: 0.56809, acc: 0.80000
[Train] Step: 7500, loss: 0.84469, acc: 0.70000
[Train] Step: 8000, loss: 0.67090, acc: 0.75000
[Train] Step: 8500, loss: 1.33463, acc: 0.60000
[Train] Step: 9000, loss: 0.66930, acc: 0.75000
[Train] Step: 9500, loss: 0.67765, acc: 0.80000
[Train] Step: 10000, loss: 1.05175

[Train] Step: 76000, loss: 0.21699, acc: 0.90000
[Train] Step: 76500, loss: 0.25512, acc: 0.95000
[Train] Step: 77000, loss: 0.71854, acc: 0.75000
[Train] Step: 77500, loss: 0.37641, acc: 0.85000
[Train] Step: 78000, loss: 0.16266, acc: 0.95000
[Train] Step: 78500, loss: 0.31521, acc: 0.90000
[Train] Step: 79000, loss: 0.13569, acc: 0.95000
[Train] Step: 79500, loss: 0.38104, acc: 0.80000
[Train] Step: 80000, loss: 0.07283, acc: 1.00000
(10000, 3072)
(10000,)
[Test] Step: 80000, acc: 0.81900
[Train] Step: 80500, loss: 0.19052, acc: 0.95000
[Train] Step: 81000, loss: 0.45169, acc: 0.85000
[Train] Step: 81500, loss: 0.28069, acc: 0.90000
[Train] Step: 82000, loss: 0.58423, acc: 0.80000
[Train] Step: 82500, loss: 0.19257, acc: 0.95000
[Train] Step: 83000, loss: 0.24692, acc: 0.90000
[Train] Step: 83500, loss: 0.39929, acc: 0.85000
[Train] Step: 84000, loss: 0.52644, acc: 0.75000
[Train] Step: 84500, loss: 0.34544, acc: 0.95000
[Train] Step: 85000, loss: 0.03198, acc: 1.00000
(10000, 3072)

In [ ]:
# train 10K -> 75%